# Load-kb-data
Notebook responsible to load docs in AI Search

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/customerSpecialist/refs/heads/main/customer-specialist/FlowDrawn.png" alt="Clean Data" style="width: 800px">
</div>

### Paths

In [13]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SimpleField, SearchableField, SearchFieldDataType
)
from azure.search.documents import SearchClient
import uuid
import os
from dotenv import load_dotenv

### Confs

In [14]:
load_dotenv()

SERVICE_NAME = os.getenv("AZURE_SEARCH_SERVICE_NAME")
INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")
API_KEY = os.getenv("AZURE_SEARCH_API_KEY")

endpoint = f"https://{SERVICE_NAME}.search.windows.net"

### Index Client

In [15]:
index_client = SearchIndexClient(endpoint=endpoint, credential=AzureKeyCredential(API_KEY))

### Create Index

In [16]:
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
    SearchableField(name="titulo", type=SearchFieldDataType.String)
]

index = SearchIndex(name=INDEX_NAME, fields=fields)

if INDEX_NAME in [idx.name for idx in index_client.list_indexes()]:
    index_client.delete_index(INDEX_NAME)

index_client.create_index(index)

print(f"Índice '{INDEX_NAME}' criado com sucesso!")

Índice 'procedimentos-kb' criado com sucesso!


### Documents

In [17]:
search_client = SearchClient(endpoint=endpoint, index_name=INDEX_NAME, credential=AzureKeyCredential(api_key))

docs = [
    {
        "id": str(uuid.uuid4()),
        "titulo": "Resetar senha de colaborador",
        "content": "Qualquer colaborador pode solicitar redefinição de senha para si mesmo sem autorização adicional."
    },
    {
        "id": str(uuid.uuid4()),
        "titulo": "Resetar senha de gerente",
        "content": "Apenas o RH pode autorizar a redefinição de senha para um gerente. Nenhum outro setor tem permissão."
    },
    {
        "id": str(uuid.uuid4()),
        "titulo": "Deletar conta",
        "content": "Nunca se deve deletar contas de usuários sem autorização por escrito do supervisor responsável."
    },
    {
        "id": str(uuid.uuid4()),
        "titulo": "Acesso de estagiários",
        "content": "Estagiários não devem receber permissões administrativas em nenhum sistema."
    }
]



### Upload Documents

In [18]:
result = search_client.upload_documents(documents=docs)

print("Documentos enviados com sucesso:")
for r in result:
    print(r)

Documentos enviados com sucesso:
{'additional_properties': {}, 'key': '3f978bc8-b323-4b3a-8322-81206e3f10e0', 'error_message': None, 'succeeded': True, 'status_code': 201}
{'additional_properties': {}, 'key': '73db56ad-1b96-4294-8e77-8a1f20232962', 'error_message': None, 'succeeded': True, 'status_code': 201}
{'additional_properties': {}, 'key': '57be629e-596c-4f57-ab82-bf363d5b6ee4', 'error_message': None, 'succeeded': True, 'status_code': 201}
{'additional_properties': {}, 'key': '8cddff32-4dca-43bf-a79a-04a1b12fbd7a', 'error_message': None, 'succeeded': True, 'status_code': 201}


### Test

In [20]:
pergunta = "Posso resetar a senha de um gerente sem aprovação?"

resultados = search_client.search(search_text=pergunta, top=3)

print("Contexto encontrado:")
for doc in resultados:
    print("-", doc['content'])

Contexto encontrado:
- Apenas o RH pode autorizar a redefinição de senha para um gerente. Nenhum outro setor tem permissão.
- Qualquer colaborador pode solicitar redefinição de senha para si mesmo sem autorização adicional.
- Nunca se deve deletar contas de usuários sem autorização por escrito do supervisor responsável.
